In [ ]:
import numpy as np
import pandas as pd
from popper.util import Settings, print_prog_score
from popper.loop import learn_solution

In [ ]:
def modify_exs_file(path_exs, content, line):
    new_content = list()

    for i in range(len(content)):
        if i == line:
            if content[line].startswith('pos'):
                new_content.append(content[line].replace('pos', 'neg'))
            elif content[line].startswith('neg'):
                new_content.append(content[line].replace('neg', 'pos'))
            else:
                new_content.append(content[line])
        else:
            new_content.append(content[i])
 
    with open(path_exs_file, 'w') as exs_file:
        exs_file.writelines(new_content)
    exs_file.close()
    return()

In [ ]:
def PADA(path, time_out):
    anomalies = list()
    analisys = list()
    cols = ['TP', 'FN', 'TN', 'FP', 'Size']
    data = Settings(kbpath = path, timeout = time_out)
    path_exs = path + '/exs.pl'
    next_iter = 1
    
    with open(path_exs, 'r') as exs_file:
        content = exs_file.readlines()
    exs_file.close()
    num_exs = len(content)

    while(next_iter):
        for i in range(num_exs + 1):
            modify_exs_file(path_exs, content, i)
            prg, res, est = learn_solution(data)
            if prg != None:
                analisys.append(res)
            else:
                analisys.append((np.NaN, np.NaN, np.NaN, np.NaN, np.NaN))
            modify_exs_file(path_exs, content, i)
        
        df = pd.DataFrame(analisys, columns=cols)
        df['Errors'] = df['FN'] + df['FP']
        anomaly = df[df.Errors == df['Errors'].min()].idxmin()['Size'] # Hay que añadir max de index -> 2 o más soluciones con = size
        if anomaly == num_exs:
            next_iter = 0
        else:
            anomalies.append(anomaly)
            modify_exs_file(path_exs, content, anomaly)
    return(anomalies)